# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- template script for analyzing modes spectra for sumatra
- downloads LHZ data needed for hw_sumatraB

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import os
import warnings

from obspy import read
from obspy import Stream
from obspy.clients.fdsn import Client
from pysep import Pysep

from lib_seis import station_map_and_table
from lib_seis import sumatra_event
from lib_seis import wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = 16, 8
plt.rcParams['lines.linewidth'] = 1

class SEED_ID_Error(Exception):
    pass

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake <br/>
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for item in event:
    print(f'{item} = {event[item]}')
print()    

### PySEP - Python Seismogram Extraction and Processing

- We will gather data for this event using the [PySEP](https://pysep.readthedocs.io/en/devel/index.html)
package.
- The PySEP package uses [ObsPy](https://docs.obspy.org/) internally to collect and handle seismic data.
- Check the webpages for details on
[data gathering](https://pysep.readthedocs.io/en/devel/autoapi/pysep/pysep/index.html#pysep.pysep.Pysep) and
[record section plotting](https://pysep.readthedocs.io/en/devel/autoapi/pysep/recsec/index.html#pysep.recsec.RecordSection)
- Check the (IRIS)[https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming] webpage for the SEED
format seismic channel naming

In [ ]:
# raw LHZ data download specifications

stations   = dict( networks                     = 'G,II,IU',
                   stations                     = '*',
                   locations                    = '*',
                   channels                     = 'LHZ' )

screening  = dict( seconds_before_ref           =  0.5 * 24 * 60 * 60,
                   seconds_after_ref            = 10.0 * 24 * 60 * 60,
                   remove_clipped               = False,
                   remove_insufficient_length   = False,
                   fill_data_gaps               = 0,
                   gap_fraction                 = 1,
                   resample_freq                = 1     )

processing = dict( remove_response              = False ) 
                   
output     = dict( log_level                    = 'INFO',
                   plot_files                   = 'map',
                   output_dir                   = 'datawf',
                   sac_subdir                   = '', 
                   overwrite_event_tag          = 'sumatra_LHZ_raw',
                   overwrite                    = False )

In [ ]:
# download data

data_dir  = f'{output["output_dir"]}/{output["overwrite_event_tag"]}'
overwrite = f'{output["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**event,**stations,**screening,**processing,**output)
    ps.run()
else:
    print('\nData directory already exists, no data will be downloaded\n')

In [ ]:
# read downloaded raw LHZ data

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# source station map for all downloaded data

event_path = f'{data_dir}/event.xml'
station_map_and_table(st, event_path)

In [ ]:
# plotting all waveforms and their spectra

plot_seismogram = True
plot_spectra    = True

# resample_freq is provided as input into pysep (see above)
Nyquist_frequency = screening['resample_freq']/2

f1 = 0.20 #mHz
f2 = 1.00 #mHz

color = ['r', 'g', 'b']

for i, tr in enumerate(st):

    if plot_seismogram:
        plt.figure()
        # replacing filled gaps (0's) with gaps (Nan's)
        plt.plot(np.where(tr.data==0.00000000, np.nan, tr.data),color=color[i%3])
        plt.xlim(0,907200)
        plt.title(f'{i+1}. {tr.id} − starting {tr.stats["starttime"]}')
        plt.show()

    if plot_spectra:
        tr1 = tr.copy()
        tr1.detrend('constant')
        tr1.detrend('linear')
        tr1.taper(max_percentage=0.2, type="cosine")
        fft_amp, fft_phase, fft_freq = wf_fft(tr1.data, Nyquist_frequency)

        plt.figure()
        plt.plot(fft_freq*1E3,fft_amp,color=color[i%3])
        plt.xlim(f1,f2)
        #plt.ylim(0,40)
        xmask = np.logical_and(fft_freq*1e3 > f1, fft_freq*1e3 < f2)
        plt.ylim(0,np.max(fft_amp[xmask])*1.1)
        plt.title(f'{i+1}. {tr1.id}')
        plt.xlabel(f'Frequency (mHz)')
        plt.ylabel(f'Amplitude (counts s)')
        plt.show()

    print('\n \n \n \n')

In [ ]:
# pick at least 20 waveforms having a useful and clean amplitude spectrum
# pick only one location for a given station

# list of stations with sensors at multiple locations
# network = G  - KIP
# network = II - ALE, ASCN, BORG, DGAR, KDAK, PFO, RPN, WRAB
# network = IU - AFI, ANMO, CASY, COLA, FURI, GUMO, HNR, INCN, KIP, QSPA
#                RAO, RAR, RCBR, SAML, SDV, SNZO, TATO, TEIG, TRIS, TRQA, WAKE

# waveform = 'network.station.location.channel'

picked_waveforms = ['G.CAN..LHZ'    ,
                    'II.NNA.00.LHZ' ,
                    'II.SUR.00.LHZ' ,
                    'IU.BBSR.00.LHZ']

In [ ]:
# plotting and listing station information corresponding to selected waveforms

st_select = Stream()

for seed_id in picked_waveforms:
    st_pick = st.select(id=seed_id)
    if not st_pick:
        raise SEED_ID_Error(f'{seed_id} not in inventory')
    st_select += st_pick

station_map_and_table(st_select, event_path)

In [ ]:
# computing and plotting the amplitude spectra of the response removed LHZ waveforms for the selected station
# locations

remove_response = True

# NOTE: you might have to rerun this cell if the execution fails

if remove_response:
    client    = Client("IRIS")
    starttime = event['origin_time'] + screening['seconds_before_ref']
    endtime   = event['origin_time'] + screening['seconds_after_ref']

    # resample_freq is provided as input into pysep (see above)
    Nyquist_frequency = screening['resample_freq']/2

    for tr_select in st_select:

        tr = tr_select.copy()
        
        tr.detrend('constant')
        tr.detrend('linear')
        tr.taper(max_percentage=0.2, type="cosine")

        fft_amp, fft_phase, fft_freq = wf_fft(tr.data, Nyquist_frequency)

        if remove_response:
            network, station, location, channel = tr.id.split('.')
            inv = client.get_stations(network=network, station=station, location=location, channel=channel,
                                      level="response", starttime=starttime, endtime=endtime)
            Ia  = inv[0][0][0].response.get_evalresp_response_for_frequencies(fft_freq, output='ACC',
                                                                              start_stage=1, end_stage=3)
            C   = fft_amp * np.exp( 1j * fft_phase )
            Xa  = np.divide(C,Ia)
            [fft_amp , fft_phase , fft_freq ] = [abs(Xa), np.angle(Xa), fft_freq]

        f1 = 0.20
        f2 = 1.00

        plt.figure()
        plt.plot(fft_freq*1E3,fft_amp)
        plt.xlim(f1,f2)
        #plt.ylim(0,40)
        xmask = np.logical_and(fft_freq*1e3 > f1, fft_freq*1e3 < f2)
        plt.ylim(0,np.max(fft_amp[xmask])*1.1)
        plt.title(f'{tr.id}')
        plt.xlabel(f'Frequency (mHz)')
        plt.ylabel(f'Amplitude (ms^-2 s)')
        plt.show()

In [ ]:
'''
from libsol_seis import sumatra_modes_waveforms
seeds_clean, seeds_cleanish, seeds_trim_clean, _ = sumatra_modes_waveforms()

for seed in seeds_trim_clean:
    seeds_trim.append(seed[0])

picked_waveforms = seeds_clean + seeds_cleanish + seeds_trim

    if tr.id in seeds_trim:
        ix = seeds_trim.index(tr.id)
        t0 = tr.stats.starttime
        t1 = seeds_trim_clean[ix][1]
        t2 = seeds_trim_clean[ix][2]
        tr.trim(t0+t1,t0+t2)

'''